### Calculate Fees

In [1]:
# Constants
TWOMBA_ENABLED = True 
TAKER_FEE_CONSTANT = 0.07
CREATORS_EARN_WHOLE_FEE_UP_TO = 1000

# Flat fees
FLAT_TRADE_FEE = 0.1
FLAT_COMMENT_FEE = 1

DPM_PLATFORM_FEE = 0.0
DPM_CREATOR_FEE = 0.0
DPM_FEES = DPM_PLATFORM_FEE + DPM_CREATOR_FEE


class Fees:
    def __init__(self, creatorFee=0, platformFee=0, liquidityFee=0):
        self.creatorFee = creatorFee
        self.platformFee = platformFee
        self.liquidityFee = liquidityFee
    
    def __repr__(self):
        return f"Fees(creatorFee={self.creatorFee}, platformFee={self.platformFee}, liquidityFee={self.liquidityFee})"
    
    def add(self, other):
        return Fees(
            self.creatorFee + other.creatorFee,
            self.platformFee + other.platformFee,
            self.liquidityFee + other.liquidityFee
        )

def addObjects(obj1, obj2):
    return obj1.add(obj2)

def getTakerFee(shares: float, prob: float) -> float:
    return TAKER_FEE_CONSTANT * prob * (1 - prob) * shares

def getFeesSplit(totalFees, previouslyCollectedFees) -> Fees:
    if TWOMBA_ENABLED:
        return Fees(0, totalFees, 0)
    
    before1k = max(0, CREATORS_EARN_WHOLE_FEE_UP_TO - previouslyCollectedFees.creatorFee)
    allToCreatorAmount = min(totalFees, before1k)
    splitWithCreatorAmount = totalFees - allToCreatorAmount
    
    return Fees(
        creatorFee=allToCreatorAmount + splitWithCreatorAmount * 0.5,
        platformFee=splitWithCreatorAmount * 0.5,
        liquidityFee=0
    )

def getFeeTotal(fees) -> float:
    return fees.creatorFee + fees.platformFee + fees.liquidityFee

def sumAllFees(fees_list) -> float:
    total_fees = Fees()
    for fee in fees_list:
        total_fees = addObjects(total_fees, fee)
    return getFeeTotal(total_fees)


### Calculate CPMM

In [4]:
class CpmmState:
    def __init__(self, pool: dict[str, float], p: float, collectedFees: Fees):
        self.pool = pool
        self.p = p
        self.collectedFees = collectedFees
        
def getCpmmProbability(pool: dict[str, float], p: float) -> float:
    YES, NO = pool['YES'], pool['NO']
    return (p * NO) / ((1 - p) * YES + p * NO)

def getCpmmProbabilityAfterBetBeforeFees(state: CpmmState, outcome: str, bet: float) -> float:
    pool, p = state.pool, state.p
    shares = calculateCpmmShares(pool, p, bet, outcome)
    YES, NO = pool['YES'], pool['NO']
    
    if outcome == "YES":
        newY, newN = YES - shares + bet, NO + bet
    else:
        newY, newN = YES + bet, NO - shares + bet
    
    return getCpmmProbability({'YES': newY, 'NO': newN}, p)

def getCpmmOutcomeProbabilityAfterBet(state: CpmmState, outcome: str, bet: float) -> float:
    newPool = calculateCpmmPurchase(state, bet, outcome)
    p = getCpmmProbability(newPool, state.p)
    return 1 - p if outcome == 'NO' else p

# before liquidty fee
def calculateCpmmShares(pool: dict[str, float], p: float, betAmount: float, betChoice: str) -> float:
    if betAmount == 0:
        return 0
    
    YES, NO = pool["YES"], pool["NO"]
    k = YES ** p * NO ** (1 - p)
    
    # https://www.wolframalpha.com/input?i=%28y%2Bb-s%29%5E%28p%29*%28n%2Bb%29%5E%281-p%29+%3D+k%2C+solve+s
    if betChoice == "YES":
        return YES + betAmount - (k * (betAmount + NO) ** (p - 1)) ** (1 / p)
    elif betChoice == "NO":
        return NO + betAmount - (k * (betAmount + NO) ** -p) ** (1 / (1 - p))

def getCpmmFees(state: CpmmState, betAmount: float, outcome: str) -> tuple:
    fee = 0
    for _ in range(10):
        betAmountAfterFee = betAmount - fee
        shares = calculateCpmmShares(state.pool, state.p, betAmountAfterFee, outcome)
        averageProb = betAmountAfterFee / shares
        fee = getTakerFee(shares, averageProb)
    
    totalFees = 0 if betAmount == 0 else fee
    fees = getFeesSplit(totalFees, state.collectedFees)
    remainingBet = betAmount - totalFees
    
    return remainingBet, totalFees, fees

def calculateCpmmSharesAfterFee(state: CpmmState, bet: float, outcome: str) -> float:
    pool, p = state.pool, state.p
    remainingBet = getCpmmFees(state, bet, outcome)[0]
    
    return calculateCpmmShares(pool, p, remainingBet, outcome)

def calculateCpmmPurchase(state: CpmmState, bet: float, outcome: str, freeFees:bool=False) -> tuple:
    pool, p = state.pool, state.p
    remainingBet, fees = (bet, Fees()) if freeFees else (getCpmmFees(state, bet, outcome)[0], getCpmmFees(state, bet, outcome)[2])
    
    shares = calculateCpmmShares(pool, p, remainingBet, outcome)
    YES, NO = pool['YES'], pool['NO']
    
    fee = fees.liquidityFee 
    
    if outcome == 'YES':
        newY, newN = YES - shares + remainingBet + fee, NO + remainingBet + fee
    else:
        newY, newN = YES + remainingBet + fee, NO - shares + remainingBet + fee    
    
    postBetPool = {'YES': newY, 'NO': newN}

    newPool, newP = (addCpmmLiquidity(postBetPool, p, fee)[0], addCpmmLiquidity(postBetPool, p, fee)[2])
    
    return shares, newPool, newP, fees

def calculateCpmmAmountToProb(state: CpmmState, prob: float, outcome: str):
    

def getCpmmLiquidity(pool: dict[str, float], p: float) -> float:
    YES, NO = pool['YES'], pool['NO']
    return YES ** p * NO ** (1 - p)

def addCpmmLiquidity(pool: dict[str, float], p: float, amount: float) -> tuple:
    prob = getCpmmProbability(pool, p)
    
    # https://www.wolframalpha.com/input?i=p%28n%2Bb%29%2F%28%281-p%29%28y%2Bb%29%2Bp%28n%2Bb%29%29%3Dq%2C+solve+p
    YES, NO = pool['YES'], pool['NO']
    numerator = prob * (amount + YES)
    denominator = amount - NO * (prob - 1) + prob * YES
    newP = numerator / denominator
    newPool = {'YES': YES + amount, 'NO': NO + amount}
    
    oldLiquidity = getCpmmLiquidity(pool, newP)
    newLiquidity = getCpmmLiquidity(newPool, newP)
    liquidity = newLiquidity - oldLiquidity
    
    return newPool, liquidity, newP


0.5470383275261324

### Test Cases